# Lab Weeks 6 and 7
### This is considered for week 6 and 7 labs, 
### For Students

In [1]:
# importing modules 
import cv2     # open cv 2 package
import numpy as np
from matplotlib import pyplot as plt



### part 1:  
#### 30 minutesfor CW1 trouble shooting, 


### Part 2: Texture Analysis

### 2-1: LBP: Local Binary Patterns
#### A. Standard LBP
1. Develop a standard LBP function
2. Try it on 
a. IMG_0054_1024bw.bmp 
b. djzam_nat_defect_002_2g_8.bmp 
c. diag_texture.bmp 
d. hor_texture.jpg 
3. If the image is color, convert it to graylevel
4. Draw the histogram of the LBP, see the differences, discuss it
5. In pparticular, compare the histogram of the LBPs of a and b, and c and d


#### B. Try to develop a radial P=16, R=3 LBP function
#### See Slide 23, c 
Then try it on 
- cat_bw.bmp
- djzam_nat_defect_002_2g_8.bmp
Plot the resulting histogram. How many bins has it got? 

### 2-2: GLCM matrices 
#### Develop a routine to compute the GLCM matrix of an image. Input parameters could be the gray-level image, 
#### number of possible gray-levels of that image, and the distance and direction between paired pixels
#### You may test your routine on the small size matrices available in slides


1. Try your GLCM routine on a few textures, e.g.
- hor_texture.jpg
2. Dont forget to convert the image to GL if it's necessary
3- Go for d=1 , and Theta= 0 
4- Apply the F1 and F2 on your GLCM matrices and see the results

## 2-3: GLCM-based Texture Analysis
1. For images below, where GL=256 for all, 
- hor_texture.jpg 
- high_spat_freq.bmp
- IMG_8636q.jpg 
2. Convert them the GL if it's necessary 
3. Apply a Gaussian lowpass filter on IMG_8636q.jpg next
4. Compute 4 GLCM matrices for distance=d=[1,5] , theta= [0,90] for each texture
5. Compute F1 and F2 for each GLCM matrix. 
6. This way, you will have a 2-feature feature vector for each GLCM matrix, 
- or a 8-feature feature vector to represent your texture
7. Compare GLCMs' [F1, F2] feature vectors with each other and discuss the results



### you may use function below to compute F1 and F2, for your texture


In [2]:
# function to extract 2 features from the glcm matrix. 
# input is the glcm matrix 
# output is maximum and energy, F1 andd F2 
# definitions in the course slides
#
def glcm_feat( g11 ):
    mxx = np.max(g11)
    enrg = np.sum(np.multiply(g11,g11))
    
    return  mxx , enrg

### 2-3: Laws Filters: A bank of small size filters to extract texture features
#### 
1. Use instructions below to creat a 9-filter 3x3 Laws filter bank 
2. Use images below, apply all Laws filters on them, compute the filter response
3. Filter responses will configure a 9-featured feature vector
- Images: 
- IMG_0054_1024bw.bmp
- 20230515_110806.jpg 
- djzam_nat_defect_002_2g_8.bmp 
- 20230324_105524gl.jpg 
4. Discuss the results

In [3]:
# Law's filters
L = np.array([1,2,1]) / 4
# or: L = np.array([1,2,1])
E = np.array([-1,0,1])
S = np.array([-1,2,-1])
llist = [L, E, S]

laws = []
for xs in llist:
    for ys in llist:
        xs= np.reshape(xs,(3,1))
        ys = np.reshape(ys,(1,3))
        L3L3 = np.matmul(xs,ys)  
        print(L3L3)
        laws.append(L3L3)


[[0.0625 0.125  0.0625]
 [0.125  0.25   0.125 ]
 [0.0625 0.125  0.0625]]
[[-0.25  0.    0.25]
 [-0.5   0.    0.5 ]
 [-0.25  0.    0.25]]
[[-0.25  0.5  -0.25]
 [-0.5   1.   -0.5 ]
 [-0.25  0.5  -0.25]]
[[-0.25 -0.5  -0.25]
 [ 0.    0.    0.  ]
 [ 0.25  0.5   0.25]]
[[ 1  0 -1]
 [ 0  0  0]
 [-1  0  1]]
[[ 1 -2  1]
 [ 0  0  0]
 [-1  2 -1]]
[[-0.25 -0.5  -0.25]
 [ 0.5   1.    0.5 ]
 [-0.25 -0.5  -0.25]]
[[ 1  0 -1]
 [-2  0  2]
 [ 1  0 -1]]
[[ 1 -2  1]
 [-2  4 -2]
 [ 1 -2  1]]



#### now in laws we have got a list of 9 filters, 3x3 each
### 
### 

### 2-4: Gabor Filters: MSMD Approaches
#### 
1. Use images below and Gabor function in the next cell, 
2. Try different parameters of the Gabor filter and see the results
- Images: 
- IMG_0054_1024bw.bmp
- 20230515_110806.jpg 
- djzam_nat_defect_002_2g_8.bmp 
- 20230324_105524gl.jpg 
4. Discuss the results

In [4]:
# Gabor filter routine, to generate a Gabor filter with given parameters, output is a Gabor filter
# you can use it later if you like
#

def gabor(sigma, theta, Lambda, psi, gamma):
    sigma_x = sigma
    sigma_y = float(sigma) / gamma

    # Bounding box
    nstds = 3  # Number of standard deviation sigma
    xmax = max(abs(nstds * sigma_x * np.cos(theta)), abs(nstds * sigma_y * np.sin(theta)))
    xmax = np.ceil(max(1, xmax))
    ymax = max(abs(nstds * sigma_x * np.sin(theta)), abs(nstds * sigma_y * np.cos(theta)))
    ymax = np.ceil(max(1, ymax))
    xmin = -xmax
    ymin = -ymax
    (y, x) = np.meshgrid(np.arange(ymin, ymax + 1), np.arange(xmin, xmax + 1))

    # Rotation
    x_theta = x * np.cos(theta) + y * np.sin(theta)
    y_theta = -x * np.sin(theta) + y * np.cos(theta)

    gb = np.exp(-.5 * (x_theta ** 2 / sigma_x ** 2 + y_theta ** 2 / sigma_y ** 2)) * np.cos(2 * np.pi / Lambda * x_theta + psi)
    return gb



In [5]:

''' 

# cv2.getGaborKernel(ksize, sigma, theta, lambda, gamma, psi, ktype)
# 2d Gabor filter for image processing, is a 2d sine wave modulated by a Gaussian envelop
# 
# -- ksize - size of gabor filter (n, n), usually it is small, e.g. 7x7 to 30x30, size of the filter
# matrix
# -- sigma - standard deviation of the gaussian function, filter bandwidth, bandwidth of that Gaussian envelop
# -- theta - orientation of the normal to the parallel stripes, we can rotate the filter, 
# direction of the 2d sine wave
# -- lambda - wavelength of the sunusoidal factor, wavelength = 1/ frequency of the filter sine wave
# -- gamma - spatial aspect ratio, 
# -- psi - phase offset, usually 0 , phase of that sine wave
# -- ktype - type and range of values that each pixel in the gabor kernel can hold,
#either 32 bit or 64 bit, float 

'''

' \n\n# cv2.getGaborKernel(ksize, sigma, theta, lambda, gamma, psi, ktype)\n# 2d Gabor filter for image processing, is a 2d sine wave modulated by a Gaussian envelop\n# \n# -- ksize - size of gabor filter (n, n), usually it is small, e.g. 7x7 to 30x30, size of the filter\n# matrix\n# -- sigma - standard deviation of the gaussian function, filter bandwidth, bandwidth of that Gaussian envelop\n# -- theta - orientation of the normal to the parallel stripes, we can rotate the filter, \n# direction of the 2d sine wave\n# -- lambda - wavelength of the sunusoidal factor, wavelength = 1/ frequency of the filter sine wave\n# -- gamma - spatial aspect ratio, \n# -- psi - phase offset, usually 0 , phase of that sine wave\n# -- ktype - type and range of values that each pixel in the gabor kernel can hold,\n#either 32 bit or 64 bit, float \n\n'

In [6]:

# building the filter kernel
g_kernel90 = cv2.getGaborKernel((21, 21), 8.0, np.pi/2, 10.0, 0.5, 0, ktype=cv2.CV_32F)

# image in gray level and convolution for 2d filtering
img = cv2.imread(im_name,0)
 
filtered_img90 = cv2.filter2D(img, cv2.CV_8UC3, g_kernel90)

# showing the input and filtered images
cv2.imshow('image', img)
cv2.imshow('filtered image, theta=90', filtered_img90)

# making the filter larger just to show that
h, w = g_kernel90.shape[:2]
g_kernel2 = cv2.resize(g_kernel90, (10*w, 10*h), interpolation=cv2.INTER_CUBIC)
cv2.imshow('gabor kernel, theta=90 (resized)', g_kernel2)


NameError: name 'im_name' is not defined